In [73]:
from pathlib import Path

In [74]:
test_text = "8A004A801A8002F478"
input_text = Path("input.txt").read_text()

In [20]:
def hex2bin(hex_str):
    return "".join([format(int(c, 16), "04b") for c in hex_str])


assert hex2bin("D2FE28") == "110100101111111000101000"
assert (
    hex2bin("38006F45291200")
    == "00111000000000000110111101000101001010010001001000000000"
)
assert (
    hex2bin("EE00D40C823060")
    == "11101110000000001101010000001100100000100011000001100000"
)

In [97]:
type("test") == str

True

In [61]:
def interpret_packet(packet):
    
    if type(packet) == str
    
    packet_dict = dict(packet_bin = packet_bin)

    version = int(packet_bin[:3], 2)
    type_id = int(packet_bin[3:6], 2)

    packet_bin = packet_bin[6:]

    packet_dict = {}
    packet_dict["version"] = version
    packet_dict["type_id"] = type_id

    if type_id == 4:

        number = ""
        keep_reading = True

        while keep_reading:

            number += packet_bin[1:5]
            keep_reading = packet_bin[0] == "1"

            packet_bin = packet_bin[5:]

        literal = int(number, 2)
        packet_dict["literal"] = literal

        return packet_dict, packet_bin

    else:

        length_type_id = packet_bin[0]
        packet_dict["length_type_id"] = length_type_id

        if length_type_id == "0":

            sub_packet_length = int(packet_bin[1:16], 2)
            packet_dict["sub_packet_length"] = sub_packet_length

            sub_packets = []

            sub_packet_bin = packet_bin[16 : 16 + sub_packet_length]
            packet_bin = packet_bin[16 + sub_packet_length :]

            while sub_packet_bin:

                sub_packet, sub_packet_bin = interpret_bin_packet(sub_packet_bin)
                sub_packets.append(sub_packet)

            packet_dict["sub_packets"] = sub_packets

            return packet_dict, packet_bin

        if length_type_id == "1":

            n_sub_packets = int(packet_bin[1:12], 2)
            packet_dict["n_sub_packets"] = n_sub_packets

            sub_packets = []

            packet_bin = packet_bin[12:]

            for i in range(n_sub_packets):

                sub_packet, packet_bin = interpret_bin_packet(packet_bin)
                sub_packets.append(sub_packet)

            packet_dict["sub_packets"] = sub_packets

            return packet_dict, packet_bin

In [63]:
def find_version_sum(packet):

    version_sum = packet["version"]

    if "sub_packets" in packet:

        for sub_packet in packet["sub_packets"]:

            version_sum += find_version_sum(sub_packet)

    return version_sum

In [60]:
interpret_bin_packet(hex2bin("38006F45291200"))

({'version': 1,
  'type_id': 6,
  'length_type_id': '0',
  'sub_packet_length': 27,
  'sub_packets': [{'version': 6, 'type_id': 4, 'literal': 10},
   {'version': 2, 'type_id': 4, 'literal': 20}]},
 '0000000')

In [62]:
interpret_bin_packet(hex2bin("EE00D40C823060"))

({'version': 7,
  'type_id': 3,
  'length_type_id': '1',
  'n_sub_packets': 3,
  'sub_packets': [{'version': 2, 'type_id': 4, 'literal': 1},
   {'version': 4, 'type_id': 4, 'literal': 2},
   {'version': 1, 'type_id': 4, 'literal': 3}]},
 '00000')

In [64]:
interpret_bin_packet(hex2bin("8A004A801A8002F478"))

({'version': 4,
  'type_id': 2,
  'length_type_id': '1',
  'n_sub_packets': 1,
  'sub_packets': [{'version': 1,
    'type_id': 2,
    'length_type_id': '1',
    'n_sub_packets': 1,
    'sub_packets': [{'version': 5,
      'type_id': 2,
      'length_type_id': '0',
      'sub_packet_length': 11,
      'sub_packets': [{'version': 6, 'type_id': 4, 'literal': 15}]}]}]},
 '000')

In [70]:
find_version_sum(interpret_bin_packet(hex2bin("8A004A801A8002F478"))[0])

16

In [72]:
def hex2sum(hex_packet):

    return find_version_sum(interpret_bin_packet(hex2bin(hex_packet))[0])


assert hex2sum("8A004A801A8002F478") == 16
assert hex2sum("620080001611562C8802118E34") == 12
assert hex2sum("C0015000016115A2E0802F182340") == 23
assert hex2sum("A0016C880162017C3686B18A3D4780") == 31

In [75]:
hex2sum(input_text)

923

# Part 2

In [93]:
def evaluate_packet(packet):

    if packet["type_id"] == 4:
        return packet["literal"]

    if packet["type_id"] == 0:
        # sum

        return sum(evaluate_packet(p) for p in packet["sub_packets"])

    if packet["type_id"] == 1:
        # prod

        prod = 1

        for p in packet["sub_packets"]:
            prod *= evaluate_packet(p)

        return prod

    if packet["type_id"] == 2:
        # min

        return min(evaluate_packet(p) for p in packet["sub_packets"])

    if packet["type_id"] == 3:
        # max

        return max(evaluate_packet(p) for p in packet["sub_packets"])

    if packet["type_id"] == 5:
        # gt

        return int(
            evaluate_packet(packet["sub_packets"][0])
            > evaluate_packet(packet["sub_packets"][1])
        )
    
    if packet["type_id"] == 6:
        # lt

        return int(
            evaluate_packet(packet["sub_packets"][0])
            < evaluate_packet(packet["sub_packets"][1])
        )
    
    if packet["type_id"] == 7:
        # eq

        return int(
            evaluate_packet(packet["sub_packets"][0])
            == evaluate_packet(packet["sub_packets"][1])
        )


assert evaluate_packet(interpret_bin_packet(hex2bin("C200B40A82"))[0]) == 3
assert evaluate_packet(interpret_bin_packet(hex2bin("04005AC33890"))[0]) == 54
assert evaluate_packet(interpret_bin_packet(hex2bin("880086C3E88112"))[0]) == 7
assert evaluate_packet(interpret_bin_packet(hex2bin("CE00C43D881120"))[0]) == 9
assert evaluate_packet(interpret_bin_packet(hex2bin("D8005AC2A8F0"))[0]) == 1
assert evaluate_packet(interpret_bin_packet(hex2bin("F600BC2D8F"))[0]) == 0
assert evaluate_packet(interpret_bin_packet(hex2bin("9C005AC2F8F0"))[0]) == 0
assert evaluate_packet(interpret_bin_packet(hex2bin("9C0141080250320F1802104A08"))[0]) == 1

In [94]:
evaluate_packet(interpret_bin_packet(hex2bin(input_text))[0])

258888628940

In [78]:
interpret_bin_packet(hex2bin("C200B40A82"))

({'version': 6,
  'type_id': 0,
  'length_type_id': '1',
  'n_sub_packets': 2,
  'sub_packets': [{'version': 6, 'type_id': 4, 'literal': 1},
   {'version': 2, 'type_id': 4, 'literal': 2}]},
 '')

In [80]:
evaluate_packet(interpret_bin_packet(hex2bin("C200B40A82"))[0])

3

In [83]:
evaluate_packet(interpret_bin_packet(hex2bin("04005AC33890"))[0])

54

In [85]:
evaluate_packet(interpret_bin_packet(hex2bin("880086C3E88112"))[0])

7